In [ ]:
import cv2
import pytesseract
import pyttsx3
import requests
import numpy as np  # Add this line to import NumPy

# Path to Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Initialize the pyttsx3 engine
engine = pyttsx3.init()

# Function to call the API for Indian currency denomination recognition
def recognize_currency(image):
    # Your API endpoint for currency recognition
    api_endpoint = 'http://127.0.0.1:5000/classify_denomination'

    # Send POST request to the API with the image data
    response = requests.post(api_endpoint, files={'image': ('image.jpg', cv2.imencode('.jpg', image)[1])})

    # Check if request was successful
    if response.status_code == 200:
        return response.text
    else:
        print("Error: Unable to recognize currency.")
        return ""

# Function to convert text to speech
def convert_text_to_speech(text):
    # Convert the text to speech
    engine.say(text)
    engine.runAndWait()

# Function to perform object detection using YOLO
def detect_objects(image):
    # Load YOLO model
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    classes = []
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getLayerNames()
    output_layers_names = net.getUnconnectedOutLayersNames()
    output_layers = [layer_names.index(layer) for layer in output_layers_names]

    # Convert image to blob
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Process detections
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)  
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected, print class name
                print(classes[class_id])



def main():
    # Open the default camera (usually the first camera)
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Unable to access camera.")
        return

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        if not ret:
            print("Error: Unable to capture frame.")
            break

        # Display the captured frame
        cv2.imshow('Live Camera', frame)

        # Check for user input to close the app, detect text, call the API, or perform object detection
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('p'):
            # Detect text when 'p' is pressed
            text = pytesseract.image_to_string(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), lang='eng')
            print("Detected Text:")
            print(text)
            convert_text_to_speech(text)
        elif key == ord('a'):
            # Call the API for currency recognition when 'a' is pressed
            currency_text = recognize_currency(frame)
            print("Currency Recognition:")
            print(currency_text)
            convert_text_to_speech(currency_text)
        elif key == ord('w'):
            # Perform object detection when 'w' is pressed
            detect_objects(frame)

    # Release the camera and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [1]:
import cv2

def detect_objects(image):
    # Load YOLO model
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    classes = []
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getLayerNames()
    output_layers_names = net.getUnconnectedOutLayersNames()
    output_layers = [layer_names.index(layer) for layer in output_layers_names]

    # Convert image to blob
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Process detections
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = scores.argmax()
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected, print class name
                print(classes[class_id])

# Load the image
image = cv2.imread("D:\TRIPS\LONAVALA\2.jpeg")

# Check if the image was loaded successfully
if image is None:
    print("Error: Unable to load image.")
    exit()

# Perform object detection on the example image
detect_objects(image)


Error: Unable to load image.


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
